# Derivation: Linear Discriminant Analysis (LDA) via SVD
How should we project high-dimensional labeled data onto a lower-dimensional space to best separate the classes? Linear Discriminant Analysis (LDA) answers this by finding directions that maximize between-class variance (spreading class means apart) while minimizing within-class variance (keeping each class compact).

> **Learning Objectives:**
>
> By the end of this lesson, you should be able to:
> * Formulate the LDA objective using within-class and between-class scatter matrices
> * Derive the generalized eigenvalue problem that maximizes Fisher's criterion
> * Develop a numerically stable SVD-based algorithm that avoids explicit matrix inversion

Let's get started!
___

## Theory: Scatter matrices and the generalized eigenvalue problem

**Notation and setup.** We have $n$ labeled samples $\{(\mathbf{x}_i, y_i)\}_{i=1}^n$, where $\mathbf{x}_i\in\mathbb{R}^{m}$ is a feature vector and $y_i$ is the class label. For each class $c$, let $n_c$ be the number of samples in $\mathcal{D}_c$ and define the class mean and overall mean:
$$
\mathbf{m}_{c} = \frac{1}{n_c}\sum_{i\,\in\,\mathcal{D}_{c}}\mathbf{x}_i, \qquad \mathbf{m} = \frac{1}{n}\sum_{i=1}^n\mathbf{x}_i.
$$

For the common binary classification case with classes $y_i\in\{-1,1\}$, we partition the data into $\mathcal{D}_{+} = \{\mathbf{x}_i : y_i=1\}$ and $\mathcal{D}_{-} = \{\mathbf{x}_i : y_i=-1\}$ with corresponding means $\mathbf{m}_{+}$ and $\mathbf{m}_{-}$.

**Scatter matrices.** The **within-class scatter** and **between-class scatter** measure two aspects of the data:
$$
\begin{align*}
\mathbf{S}_w &= \sum_{c}\sum_{i\,\in\,\mathcal{D}_c}(\mathbf{x}_i-\mathbf{m}_{c})(\mathbf{x}_i-\mathbf{m}_{c})^{\top},\\[4pt]
\mathbf{S}_b &= \sum_{c} n_c(\mathbf{m}_{c}-\mathbf{m})(\mathbf{m}_{c}-\mathbf{m})^{\top}.
\end{align*}
$$
$\mathbf{S}_w$ quantifies within-class covariance (how tight each class is), while $\mathbf{S}_b$ quantifies between-class separation (how spread apart class means are). Together: $\mathbf{S}_t = \mathbf{S}_w + \mathbf{S}_b$.

**Fisher's criterion.** To find a projection direction $\mathbf{w}$ that best separates classes, we look for one-dimensional projections $z = \mathbf{w}^{\top}\mathbf{x}$ that __maximizes__ the ratio (Fisher's criterion):
$$
J(\mathbf{w}) = \frac{\mathbf{w}^{\top}\mathbf{S}_b\mathbf{w}}{\mathbf{w}^{\top}\mathbf{S}_w\mathbf{w}}.
$$

To find the optimal $\mathbf{w}$, impose the constraint $\mathbf{w}^{\top}\mathbf{S}_w\mathbf{w}=1$ (normalization with respect to within-class scatter) and use Lagrange multipliers:
$$
\mathcal{L}(\mathbf{w},\lambda)=\mathbf{w}^{\top}\mathbf{S}_b\mathbf{w}-\lambda(\mathbf{w}^{\top}\mathbf{S}_w\mathbf{w}-1).
$$
where $\mathcal{L}(\cdot)$ is the Lagrangian function and $\lambda$ is the Lagrange multiplier. Taking the gradient with respect to $\mathbf{w}$ and setting $\nabla_\mathbf{w}\mathcal{L}=0$ gives:
$$
\begin{align*}
\nabla_\mathbf{w}\mathcal{L} &= (\mathbf{S}_b + \mathbf{S}_b^{\top})\mathbf{w} - \lambda(\mathbf{S}_w + \mathbf{S}_w^{\top})\mathbf{w}\quad(\mathbf{S}_{b}\;\text{and}\mathbf{S}_{w}\;\text{symmetric}) \\
&= 2\mathbf{S}_b\mathbf{w} - 2\lambda\mathbf{S}_w\mathbf{w}\\
&= \mathbf{S}_b\mathbf{w} - \lambda\mathbf{S}_w\mathbf{w}
\end{align*}
$$
the __generalized eigenvalue problem__:
$$
\boxed{\mathbf{S}_b\mathbf{w} = \lambda\mathbf{S}_w\mathbf{w}},
$$
At the optimum, the generalized eigenvalue $\lambda$ equals the maximum Fisher criterion value: $\lambda_{\max} = \max J(\mathbf{w})$.

**Multiple discriminant directions.** For multi-class (beyond binary) problems, one direction is rarely sufficient. We extend the criterion by collecting $r$ projection vectors as $\mathbf{W}=[\mathbf{w}_1,\dots,\mathbf{w}_r]\in\mathbb{R}^{m\times r}$ and optimizing:
$$
J(\mathbf{W}) = \mathrm{tr}\left[\left(\mathbf{W}^{\top}\mathbf{S}_w\mathbf{W}\right)^{-1}\left(\mathbf{W}^{\top}\mathbf{S}_b\mathbf{W}\right)\right].
$$

The optimal columns still satisfy the same generalized eigenvalue equation, solved sequentially: $\mathbf{w}_1$ is the top eigenvector, $\mathbf{w}_2$ the next, and so on.

___

## Computing LDA: The SVD approach

We've now formulated the problem: find eigenvectors of $\mathbf{S}_b\mathbf{w} = \lambda\mathbf{S}_w\mathbf{w}$. A naive approach is to invert the matrix $\mathbf{S}_w$: which allows us to rewrite the generalized eigenvalue problem as $(\mathbf{S}_w^{-1}\mathbf{S}_b)\mathbf{w} = \lambda\mathbf{w}$ and compute eigenvectors. 

> __The wrinkle__: But this fails when $\mathbf{S}_w$ is singular (which happens whenever $m > n$ or when classes are low-rank). Direct inversion amplifies numerical errors and may not even be possible.

Instead, we use **SVD to construct a stable whitening transform**. This replaces the ill-conditioned inversion with a well-conditioned change of variables.

### Intuition: Whitening
Imagine your data classes look like elongated ellipsoids, some directions have high variance, others low. The within-class scatter $\mathbf{S}_w$ encodes this shape. **The core insight**: if we could __reshape these ellipsoids__ into spheres (all directions having equal variance), the problem becomes much simpler.

> **What whitening does geometrically:**
>
> - **Before whitening**: Classes might be stretched along certain axes, compressed along others. Finding the best separation direction must account for this unequal spread.
> - **After whitening**: We transform the space so that within-class variance becomes uniform in all directions—like inflating a football into a basketball. Now we can focus purely on where the class centers are, not how the classes are shaped.

Whitening normalizes out the within-class covariance structure. Once the within-class scatter looks like an identity matrix (spherical), the generalized eigenvalue problem $\mathbf{S}_b\mathbf{w} = \lambda\mathbf{S}_w\mathbf{w}$ reduces to a standard one: just find directions where the between-class scatter is largest.

**Why SVD?** The SVD gives us the natural coordinate system for this transformation, it reveals the principal directions of within-class variance and how much to scale each one to make the scatter spherical.

### Mathematics: How We Whiten
Start by collecting all data points centered by their class mean (where $\mathbf{m}_{y_i}$ is the mean of the class that sample $i$ belongs to). This gives us matrix $\mathbf{H}$ where each column is $\mathbf{x}_i-\mathbf{m}_{y_i}$:
$$
\mathbf{H} = [\mathbf{x}_1-\mathbf{m}_{y_1},\dots,\mathbf{x}_n-\mathbf{m}_{y_n}]\in\mathbb{R}^{m\times n}.
$$
The within-class scatter is $\mathbf{S}_w = \mathbf{H}\mathbf{H}^{\top}$, which sets us up for SVD. Decompose $\mathbf{H}$ using the thin SVD:
$$
\mathbf{H} = \mathbf{U}\mathbf{\Sigma}\mathbf{V}^{\top},\qquad \mathbf{U}\in\mathbb{R}^{m\times r_w},\quad \mathbf{\Sigma}\in\mathbb{R}^{r_w\times r_w},\quad r_w = \mathrm{rank}(\mathbf{H}).
$$
This gives $\mathbf{S}_w = \mathbf{U}\mathbf{\Sigma}^2\mathbf{U}^{\top}$, where $\mathbf{U}$ contains the principal axes of within-class variance and $\mathbf{\Sigma}^2$ tells us the variance along each axis. The whitening transformation $\mathbf{Z} = \mathbf{\Sigma}^{-1}\mathbf{U}^{\top}$ rotates to principal axes ($\mathbf{U}^{\top}$) and rescales each axis ($\mathbf{\Sigma}^{-1}$) to unit variance. In the whitened space, $\mathbf{Z}\mathbf{S}_w\mathbf{Z}^{\top} = \mathbf{I}$.

For the between-class scatter, we can factor it as $\mathbf{S}_b = \mathbf{M}\mathbf{M}^{\top}$, where $\mathbf{M}=[\sqrt{n_1}(\mathbf{m}_{1}-\mathbf{m}),\dots,\sqrt{n_k}(\mathbf{m}_{k}-\mathbf{m})]\in\mathbb{R}^{m\times k}$ collects the weighted deviations of each class mean from the overall mean $\mathbf{m}$. Instead of solving for $\mathbf{w}$ directly, we work in the whitened space. Define $\mathbf{a}$ as the projection direction in whitened coordinates, related to the original by $\mathbf{w}=\mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a}$. Substituting into $\mathbf{S}_b\mathbf{w} = \lambda\mathbf{S}_w\mathbf{w}$ gives:
$$
\mathbf{S}_b(\mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a}) = \lambda(\mathbf{U}\mathbf{\Sigma}^2\mathbf{U}^{\top})(\mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a}).
$$

Simplify step by step:
$$
\begin{align*}
\lambda(\mathbf{U}\mathbf{\Sigma}^2\mathbf{U}^{\top})(\mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a}) &= \lambda\mathbf{U}\mathbf{\Sigma}^2(\mathbf{U}^{\top}\mathbf{U})\mathbf{\Sigma}^{-1}\mathbf{a} \\
&= \lambda\mathbf{U}\mathbf{\Sigma}^2\mathbf{I}\mathbf{\Sigma}^{-1}\mathbf{a} \quad (\text{since } \mathbf{U}^{\top}\mathbf{U} = \mathbf{I})\\
&= \lambda\mathbf{U}\mathbf{\Sigma}\mathbf{a}.
\end{align*}
$$
So the equation becomes:
$$
\mathbf{S}_b(\mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a}) = \lambda\mathbf{U}\mathbf{\Sigma}\mathbf{a}.
$$
Multiply both sides on the left by $\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}$:
$$
\begin{align*}
\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}\mathbf{S}_b(\mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a}) &= \lambda\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}\mathbf{U}\mathbf{\Sigma}\mathbf{a} \\
\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}\mathbf{S}_b\mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a} &= \lambda\mathbf{\Sigma}^{-1}\mathbf{I}\mathbf{\Sigma}\mathbf{a} \quad (\text{since } \mathbf{U}^{\top}\mathbf{U} = \mathbf{I})\\
\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}\mathbf{S}_b\mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a} &= \lambda\mathbf{a}.
\end{align*}
$$
This yields the standard eigenvalue problem:
$$
\boxed{(\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}\mathbf{S}_b\mathbf{U}\mathbf{\Sigma}^{-1})\mathbf{a} = \lambda\mathbf{a}}.
$$
To avoid recomputing this full matrix expression, define $\mathbf{B}=\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}\mathbf{M}$ as the whitened between-class matrix. Then $\mathbf{B}\mathbf{B}^{\top} = \mathbf{\Sigma}^{-1}\mathbf{U}^{\top}\mathbf{M}\mathbf{M}^{\top}\mathbf{U}\mathbf{\Sigma}^{-1} = \mathbf{\Sigma}^{-1}\mathbf{U}^{\top}\mathbf{S}_b\mathbf{U}\mathbf{\Sigma}^{-1}$, so we solve $\mathbf{B}\mathbf{B}^{\top}\mathbf{a}=\lambda\mathbf{a}$ for the eigenvectors $\mathbf{a}$. We then transform back via $\mathbf{w} = \mathbf{U}\mathbf{\Sigma}^{-1}\mathbf{a}$ to get discriminant directions in the original coordinate system.

___

## SVD-based LDA algorithm
__Initialization__: Given labeled data $\{(\mathbf{x}_i,y_i)\}_{i=1}^n$ with $k$ classes, compute class means $\mathbf{m}_{c}$, overall mean $\mathbf{m}$, and class counts $n_c$. Choose a nonnegative tolerance $\tau$ for truncating small singular values (e.g., $\tau\approx 0$ or a small multiple of machine precision). Note: LDA can produce at most $\min(r_w, k-1)$ discriminant directions, where $r_w$ is the rank of the within-class centered matrix.

- Build the within-class centered matrix $\mathbf{H}=[\mathbf{x}_i-\mathbf{m}_{y_i}]$.
- Compute the thin SVD of the within-class centered matrix: $\mathbf{H}=\mathbf{U}\mathbf{\Sigma}\mathbf{V}^{\top}$.
- Form the between-class mean matrix $\mathbf{M}=[\sqrt{n_1}(\mathbf{m}_{1}-\mathbf{m}),\dots,\sqrt{n_k}(\mathbf{m}_{k}-\mathbf{m})]$ so that $\mathbf{S}_b=\mathbf{M}\mathbf{M}^{\top}$.
- Apply the whitening transformation to the between-class matrix: $\mathbf{B}=\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}\mathbf{M}$. This transforms the between-class mean deviations into the whitened space. 

  > __Practical note:__
  > 
  > In practice, use the Moore-Penrose pseudoinverse $\mathbf{\Sigma}^{+}$ instead of $\mathbf{\Sigma}^{-1}$. For the diagonal matrix $\mathbf{\Sigma} = \text{diag}(\sigma_1, \dots, \sigma_r)$, define:
  > $$\mathbf{\Sigma}^+ = \text{diag}\left(\frac{1}{\sigma_1}, \frac{1}{\sigma_2}, \dots, \frac{1}{\sigma_r}\right) \quad \text{where} \quad \frac{1}{\sigma_i} = \begin{cases} 1/\sigma_i & \text{if } \sigma_i > \tau \\ 0 & \text{if } \sigma_i \leq \tau \end{cases}.$$
  > This avoids dividing by tiny singular values, which would amplify numerical errors.
- Solve the reduced eigenproblem: $\mathbf{B}\mathbf{B}^{\top}\mathbf{a}=\lambda\mathbf{a}$ (using QR iteration or another eigenvalue solver) and order eigenpairs by descending $\lambda$.
- Recover discriminant directions: $\mathbf{w}=\mathbf{U}\mathbf{\Sigma}^{+}\mathbf{a}$ (using the pseudoinverse for consistency) and keep the top $r$ vectors.

__Output__: Return the LDA projection matrix $\mathbf{W}_r=[\mathbf{w}_1,\dots,\mathbf{w}_r]$ and the associated eigenvalues $\lambda_1\ge\cdots\ge\lambda_r$.

___

## Summary
LDA reduces to a generalized eigenvalue problem that can be solved stably with SVD.

> __Key Takeaways:__
>
> * LDA maximizes Fisher's criterion—the ratio of between-class to within-class scatter—yielding the generalized eigenvalue problem $\mathbf{S}_b\mathbf{w}=\lambda\mathbf{S}_w\mathbf{w}$, where at the optimum $\lambda_{\max} = \max J(\mathbf{w})$.
> * The thin SVD of the within-class centered data matrix $\mathbf{H}$ provides $\mathbf{S}_w = \mathbf{U}\mathbf{\Sigma}^2\mathbf{U}^{\top}$, enabling a numerically stable solution that naturally handles singular scatter matrices without explicit inversion.
> * Whitening via $\mathbf{Z}=\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}$ transforms the generalized eigenproblem into a standard eigenproblem in reduced space: $\left(\mathbf{\Sigma}^{-1}\mathbf{U}^{\top}\mathbf{S}_b\mathbf{U}\mathbf{\Sigma}^{-1}\right)\mathbf{a} = \lambda\mathbf{a}$.

___